# In This Demo

We explore the differnent evaluation modes that `Model` classes can now use, as well as the interface for switching between them. We will also briefly explore the `OperatorCollection` formalism which underlies the ease of switching evaluation modes, and which should also greatly simplify the process of developing new evaluation modes. 

In [1]:
import numpy as np
from qiskit_dynamics.models import (
    HamiltonianModel,
    LindbladModel,
    rotating_wave_approximation,
)
from qiskit_dynamics.signals import Signal

We begin by setting up a `Model`:

In [2]:
Omega_0 = 0.5
tau = 3
omega_0 = 1
nu = 1/(2*np.pi) #2\pi\nu = \omega_0

Omega = lambda t: Omega_0 * np.exp(-t**2/(2*tau**2))
drive_signal = Signal(Omega,nu,0)

drive_operator = np.array([[[0,-1],[-1,0]]])
drift_operator = omega_0*np.array([[1,0],[0,-1]])

model = HamiltonianModel(drive_operator,drift=drift_operator,signals=[drive_signal])

print(model.evaluate(0))

[[0.-1.j  0.+0.5j]
 [0.+0.5j 0.+1.j ]]


Note that by default, the Model class evaluates everything using dense arrays. While this is very useful for e.g. `jax` compilation or differentiation, in many cases, we may wish to have our Hamiltonian evaluate itself using sparse matrices. For this, we may either call `Model.set_evaluation_mode` or pass the keyword `evaluation_mode = (...)` in the constructor. 

In [3]:
model.set_evaluation_mode("sparse")

print(model.evaluate(0))

  (0, 0)	-1j
  (0, 1)	0.5j
  (1, 0)	0.5j
  (1, 1)	1j


These evaluation modes are compatible with rotating frames—though, in the case of sparse matrices, only diagonal frames will yield sparse generators. 

In [4]:
model.rotating_frame = np.array([[1,0], [0, -1]])
print(model(0))

[[0.-0.j  0.+0.5j]
 [0.+0.5j 0.-0.j ]]


This is compatible with the rotating wave approximation in a user-friendly way, where the RWA preserves the evaluation mode:

In [5]:
rwa_model = rotating_wave_approximation(model, cutoff_freq=1/4)
print(np.round(rwa_model(0),6))

[[ 0.-0.j    0.+0.25j]
 [-0.+0.25j  0.-0.j  ]]


We can also add dissipative channels with `LindbladModel`, which has its own set of evaluation modes, listed below: 

In [6]:
model.rotating_frame = np.array([1,-1])

dis_op = np.array([[[0,0],[1,0]]])
rho = np.diag([1,0]) # excited state
dis_model = LindbladModel.from_hamiltonian(model, dissipator_operators=dis_op, dissipator_signals=[1], evaluation_mode="dense")

print(help(dis_model.set_evaluation_mode))

Help on method set_evaluation_mode in module qiskit_dynamics.models.lindblad_models:

set_evaluation_mode(new_mode: str) method of qiskit_dynamics.models.lindblad_models.LindbladModel instance
    Sets evaluation mode.
    Args:
        new_mode: new mode for evaluation. Supported modes are:
            'dense': Stores Hamiltonian and dissipator terms as dense
                   Array types.
            'dense_vectorized': Stores the Hamiltonian and dissipator
                terms as (dim^2,dim^2) matrices that acts on a vectorized
                density matrix by left-multiplication. Allows for direct evaluate generator.
            'sparse': Like dense, but stores Hamiltonian components with
                `csr_matrix` types. Outputs will be dense if a 2d frame operator is
                used. Not compatible with jax.
    Raises:
        NotImplementedError: If a mode other than one of the above is specified.

None


With `evaluation_mode = "dense"`, things work as you might expect: the Lindbladian is evaluated purely using dense matrices: 

In [7]:
dis_model.set_evaluation_mode('dense')
print(dis_model.evaluate_hamiltonian(0))
print(dis_model(0,rho))

[[ 0. +0.j -0.5+0.j]
 [-0.5+0.j  0. +0.j]]
[[-1.+0.j   0.-0.5j]
 [ 0.+0.5j  1.+0.j ]]


When we work with `evaluation_mode = "sparse"`, the internal computation is done using sparse matrices (which can yield a substantial speed-up), but the final Lindbladian is still a dense matrix. This is primarily because many differential equation solvers require that the time derivative $\dot{\rho}$ still be a dense matrix.

In [8]:
dis_model.set_evaluation_mode('sparse')
print(dis_model.evaluate_hamiltonian(0))
print(dis_model(0,rho))

  (0, 1)	(-0.5+0j)
  (1, 0)	(-0.5+0j)
[[-1.+0.j   0.-0.5j]
 [ 0.+0.5j  1.+0.j ]]


Using either of these two evaluation modes, it is impossible for us to evaluate a single generator s.t. $\dot{\rho} = G(t)\rho$. If we wish to evaluate a generator on its own, we need to switch to using vectorized evaluation. When we evaluate the Lindbladian using vectorized evaluation (with column-stacking convention), we may evaluate a generator independent of $\rho$ itself:

In [9]:
rho_vec = rho.flatten(order="F") # vectorized operator using column stacking convention
dis_model.set_evaluation_mode('dense_vectorized')
print(dis_model(0))
print(dis_model(0) @ rho_vec)
print(dis_model(0,rho_vec))

[[-1. +0.j   0. +0.5j  0. -0.5j  0. +0.j ]
 [ 0. +0.5j -0.5+0.j   0. +0.j   0. -0.5j]
 [ 0. -0.5j  0. +0.j  -0.5+0.j   0. +0.5j]
 [ 1. +0.j   0. -0.5j  0. +0.5j  0. +0.j ]]
[-1.+0.j   0.+0.5j  0.-0.5j  1.+0.j ]
[-1.+0.j   0.+0.5j  0.-0.5j  1.+0.j ]


# Under the Hood: `OperatorCollection`

In order to enable these different evaluation modes, the actual implementaiton of evaluating the RHS of some differential equation has been abstracted away from the `Model` classes. Now, every `Model` stores an `OperatorCollection`, e.g. `DenseOperatorCollection`, `SparseLindbladCollection`, etc. When we call `Model.evaluate_rhs()`, the model performs evaluation in four steps: 

1. Pass state to `RotatingFrame` for pre-rotation (if applicable, e.g. $|\psi\rangle\to e^{-iHt}|\psi\rangle$) and basis change (if applicable).

2. Get Signal values from `SignalList` object.

3. Pass Signal values and state (if applicable) to `OperatorCollection` to evaluate model.

4. Pass model to `Rotatingframe` for post-rotation (if applicable, e.g. $|\psi\rangle \to e^{iHt}|\psi\rangle$) and basis change (if applicable).

Because each of these steps are completely modular, changing how a `Model` performs evaluation is as straightforward as swapping out its `OperatorCollection`. Likewise, developing support for a new evaluation mode is quite straightforward, requiring a developer only to write a new `OperatorCollection` class. Because these `OperatorCollections` don't need to worry about basis transformations, signals, frames, etc., they can be extremely simple objects, with most of the work involved in optimizing their `evaluate` and `evaluate_rhs` methods. This makes them quite straightforward to write.

In [10]:
sig_vals = model.signals(0)

model.set_evaluation_mode("dense")
operator_collection = model._operator_collection
print(type(operator_collection))
print(operator_collection.evaluate(sig_vals))

model.set_evaluation_mode("sparse")
operator_collection = model._operator_collection
print(type(operator_collection))
print(operator_collection.evaluate(sig_vals))

<class 'qiskit_dynamics.models.operator_collections.DenseOperatorCollection'>
[[ 0. +0.j -0.5+0.j]
 [-0.5+0.j  0. +0.j]]
<class 'qiskit_dynamics.models.operator_collections.SparseOperatorCollection'>
  (0, 1)	(-0.5+0j)
  (1, 0)	(-0.5+0j)
